In [ ]:
import gym
import gym_fish
import numpy as np
from stable_baselines3 import PPO, SAC
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')



gpuId = 0
env = gym.make('koi-cruising-v0', gpuId=gpuId)

model = PPO.load("best_model.zip", env=env)
print("model loaded")

i = 0
obs = env.reset()

distance=5


x = np.random.uniform(0.4,distance,size=(1,))
y=np.random.uniform(0,0,size=(1,))
z=(np.abs(distance**2 - x**2))**0.5
goal_pose_ = np.concatenate((x,y, z)).reshape((3, ))

env.goal_pos=goal_pose_

print(env.goal_pos, env.body_xyz)

# Open the file in write mode
with open(f"{distance}_results.txt", "a") as f:
    error_history = []

    while np.linalg.norm(env.goal_pos - env.body_xyz) > 0.25:
        action, states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        error = np.linalg.norm(env.goal_pos - env.body_xyz)
        print(f"output{i}.png\terror: {error}")

        # Append the current error to the error history
        error_history.append(error)

        # Ensure the error history doesn't grow indefinitely
        if len(error_history) > 20:
            error_history.pop(0)

        # Check if the error is increasing for the last 20 steps
        if len(error_history) == 20 and all(x < y for x, y in zip(error_history[:-1], error_history[1:])):
            print("Error has been increasing for the last 20 steps.")
            f.write("Error has been increasing for the last 20 steps.\n")
            break

        i += 1
  #      if error > 3:
   #         break

        if i >= 400:
           break

    f.write(f"{error}, {i}, {env.goal_pos}\n")

